# Set Variables

In [ ]:
top_k = 20

# -------- method_way
method_way = "c" # a, b, c

if method_way == "a":
    bug_localization_accuracy = [1] # main data - th

elif method_way == "b":
    bug_localization_accuracy = [1] # main data - th

else:
    bug_localization_accuracy = [0.4, 0.5, 0.6, 0.7, 0.8]


# -------- dataset
# software_name = "Camel"
# software_name = "CloudStack"
# software_name = "Geode"
software_name = "Hbase"

In [ ]:
print(bug_localization_accuracy)

[0.4, 0.5, 0.6, 0.7, 0.8]


In [ ]:
if software_name == "Camel":
    dataset_file_name = "Camel_Bugs - v.01"
    
elif software_name == "CloudStack":
    dataset_file_name = "CloudStack_Bugs - v.01"
    
elif software_name == "Geode":
    dataset_file_name = "Geode_Bug - v.01"
    
else:
    dataset_file_name = "Hbase_Bugs - v.03"

# Google Colab

In [ ]:
# load data from google drive
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
!ls "/content/gdrive/My Drive/"

Mounted at /content/gdrive
'Colab Notebooks'


In [ ]:
# project folder path
project_folder = "gdrive/MyDrive/Colab Notebooks/paper/"

# data folder path
data_folder = "00- My Data/two-phase method/"

# dataset folder path
dataset_folder = software_name + "/"

# output folder path
output_folder = "01- Jupyter Notebook/80- two-phase method/00. Output/"

# Libs

In [ ]:
import random

import pandas as pd

# Config

In [ ]:
mypaths = {
    "data": {
        "dataset_bugs": project_folder + data_folder + dataset_folder + dataset_file_name + ".csv",
        "dataset_bugs_bla": project_folder + data_folder + dataset_folder + "second/k={}/bla={}/" + "{} _ k={}, bla={}.csv"
    }
}

preprocessing_params = {
    "data": {
        "dataset_bugs": {
            # time_stamp is the time that we change the class for a bug fixing.
            # id stands for identifier
            "columns_name":   ["bug_report_id", "class_id", "time_stamp", "label"],
            "columns_dtype" : {0: "int64", 1: "int64", 2: "int64", 3:"int32"}
        }
    }, 
    "top_k": top_k
}

# Read Files

In [ ]:
df_main = pd.read_csv(
    mypaths["data"]["dataset_bugs"], 
    names=preprocessing_params["data"]["dataset_bugs"]["columns_name"], 
    dtype=preprocessing_params["data"]["dataset_bugs"]["columns_dtype"], 
    header=None
)

# MyDataset

In [ ]:
class MyDatasetSecond():
    
    def __init__(self, df_main, columns_name, columns_dtype, bug_localization_accuracy, top_k=5):
        self.df_main = df_main.copy()
        self.bla = bug_localization_accuracy
        self.top_k = top_k
        
        self.cIDs = set(self.df_main["class_id"].to_list()) # cIDs: class ids
        
        self.df_data = {} # [df1, df2, ...]
        self.brIDs_num_ft = {} # {"0.50": {"true": 400, "false":50}, ...}
        self.brIDs_num_total = None
        
        for bla in self.bla:
            temp = {}
            for cn, cd in zip(columns_name, columns_dtype.values()):
                temp[cn] = pd.Series(dtype=cd)
            tempEmptyDF = pd.DataFrame(temp)
            self.df_data[bla] = tempEmptyDF
            
            self.brIDs_num_ft[bla] = {
                "true": None,
                "false": None
            }
    
    
    # brids: bug report ids
    # cids: class ids
    def select_brids_replace_cids(self):
        
        # -------- select_bug_reports
        # randomly select bug reports whose class ids will be replaced
        dfg_brIDs = self.df_main.groupby("bug_report_id") # brIDs: bug report ids
        self.brIDs_num_total = len(dfg_brIDs) # number of bug reports
        brIDs = list(dfg_brIDs.groups.keys())
        
        for bla in self.bla:
            bla_false = 1 - bla
            self.brIDs_num_ft[bla]["false"] = round(self.brIDs_num_total * bla_false)
            self.brIDs_num_ft[bla]["true"] = self.brIDs_num_total - self.brIDs_num_ft[bla]["false"]
            # random.sample: selects elements without replacement
            brIDs_false = random.sample(brIDs, self.brIDs_num_ft[bla]["false"])
            
            df_brIDs_false = dfg_brIDs.filter(lambda g: brIDs_false.count(g.name) > 0)
            df_brIDs_false.groupby("bug_report_id").apply(lambda g: self.replacement_false_(g, bla))
            
            df_brIDs_true = dfg_brIDs.filter(lambda g: brIDs_false.count(g.name) == 0)
            df_brIDs_true.groupby("bug_report_id").apply(lambda g: self.replacement_true_(g, bla))
    
    
    def replacement_false_(self, g, bla):
        bug_cids_true = g["class_id"].to_list()
        num_bug_cids_true = len(bug_cids_true)
        bug_cids_true_set = set(bug_cids_true)
        bug_cids_false = list(self.cIDs - bug_cids_true_set)
        if num_bug_cids_true > self.top_k:
            # random.choices: selects elements that can be duplicated
            bug_cids_false_selected = random.choices(bug_cids_false, k=self.top_k)
            bug_cids_index = random.sample(list(range(num_bug_cids_true)), k=self.top_k)
            temp = self.df_main.loc[g.index[bug_cids_index], :].copy()
            temp["class_id"] = bug_cids_false_selected
        else:
            bug_cids_false_selected = random.choices(bug_cids_false, k=len(g))
            temp = self.df_main.loc[g.index, :].copy()
            temp["class_id"] = bug_cids_false_selected
            
        self.df_data[bla] = self.df_data[bla].append(temp, ignore_index=True)
    
    
    def replacement_true_(self, g, bla):
        bug_cids_true = g["class_id"].to_list()
        num_bug_cids_true = len(bug_cids_true)
        if num_bug_cids_true > self.top_k:
            bug_cids_index = random.sample(list(range(num_bug_cids_true)), k=self.top_k)
            temp = self.df_main.loc[g.index[bug_cids_index], :].copy()
        else:
            temp = self.df_main.loc[g.index, :].copy()
            
        self.df_data[bla] = self.df_data[bla].append(temp, ignore_index = True)
    
    def free_memory(self):
        self.df_main = None
        self.cIDs = None

## obj

In [ ]:
print("second")
md = MyDatasetSecond(
    df_main,
    preprocessing_params["data"]["dataset_bugs"]["columns_name"], 
    preprocessing_params["data"]["dataset_bugs"]["columns_dtype"], 
    bug_localization_accuracy, 
    top_k=preprocessing_params["top_k"]
)

second


In [ ]:
md.select_brids_replace_cids()
md.free_memory()

In [ ]:
print(len(md.df_data))
md.df_data

5


{0.4:        bug_report_id  class_id  time_stamp  label
 0              41737    127912  1583149803      0
 1              41737    125458  1583149803      0
 2              41737    120384  1583149803      0
 3              41737    128410  1583149803      0
 4              41737    125502  1583149803      0
 ...              ...       ...         ...    ...
 43822          53953    130964  1285307201      0
 43823          53953    131325  1285307201      0
 43824          53953    131110  1285307201      0
 43825          53953    131286  1285307201      0
 43826          53953    130678  1285307201      0
 
 [43827 rows x 4 columns],
 0.5:        bug_report_id  class_id  time_stamp  label
 0              41737    125548  1583149803      0
 1              41737    131190  1583149803      0
 2              41737    123720  1583149803      0
 3              41737    129895  1583149803      0
 4              41737    126774  1583149803      0
 ...              ...       ...         ...

# save

In [ ]:
for bla, df in md.df_data.items():
    print("-" * 10, bla , "-" * 10)
    print(df)
    print(len(df.groupby("bug_report_id")))
    print(len(df))
    full_path = mypaths["data"]["dataset_bugs_bla"].format(top_k, bla, dataset_file_name, top_k, bla)
    print(full_path)
    df.to_csv(full_path, header=False, index=False)

---------- 0.4 ----------
       bug_report_id  class_id  time_stamp  label
0              41737    127912  1583149803      0
1              41737    125458  1583149803      0
2              41737    120384  1583149803      0
3              41737    128410  1583149803      0
4              41737    125502  1583149803      0
...              ...       ...         ...    ...
43822          53953    130964  1285307201      0
43823          53953    131325  1285307201      0
43824          53953    131110  1285307201      0
43825          53953    131286  1285307201      0
43826          53953    130678  1285307201      0

[43827 rows x 4 columns]
9201
43827
gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/Hbase/second/k=20/bla=0.4/Hbase_Bugs - v.03 _ k=20, bla=0.4.csv
---------- 0.5 ----------
       bug_report_id  class_id  time_stamp  label
0              41737    125548  1583149803      0
1              41737    131190  1583149803      0
2              41737    123720